# 读写文件  
有时我们希望保
存训练的模型，以备将来在各种环境中使用（比如在部署中进行预测）。此外，当运行一个耗时较长的训练过程时，最佳的做法是定期保存中间结果，以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。因此，现在是时候学习如何加载和存储权重向量和整个模型了。  

### 1. 加载和保存张量  

In [5]:
import torch
from torch import nn 
from torch.nn import functional as F

# 单个张量
x = torch.arange(4)
torch.save(x, 'x-file') # 当前工作目录下寻找或创建文件'x-file'
x2 = torch.load('x-file') # 加载到内存
print(x2)

# 张量列表
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
print(x2, y2)

# 张量字典
mydict = {'x' : x, 'y' : y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

tensor([0, 1, 2, 3])
tensor([0, 1, 2, 3]) tensor([0., 0., 0., 0.])


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 2. 加载和保存模型参数  
深度学习框架提供了内置函数来保存和加载整个网络。需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。因此，为了恢复模型，我们需要用代码生成架构，然后从磁盘加载参数。  

In [6]:
class MLP(nn.Module):
    """两层感知机"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

# 将模型的参数存储在一个叫做“mlp.params”的文件中
torch.save(net.state_dict(), 'mlp.params')

# 为了恢复模型，实例化一个原始MLP，然后读取存储的参数
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [7]:
# 由于两个实例具有相同的模型参数，在输入相同的X时，两个实例的计算结果应该相同
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])